In [ ]:
import pandas as pd
from ehrmonize import EHRmonize

In [ ]:
models = [
    'gpt-3.5-turbo',
    # 'gpt-4', -- works well though
    'gpt-4o',
    # 'meta.llama2-13b-chat-v1',
    # 'meta.llama2-70b-chat-v1',
    'meta.llama3-70b-instruct-v1:0',
    'anthropic.claude-v2:1',
    # 'anthropic.claude-instant-v1', -- works well though
    # 'mistral.mistral-7b-instruct-v0:2',
    'mistral.mixtral-8x7b-instruct-v0:1',
]

temperatures = [
    0.0,
    0.1,
    0.2,
    0.5
]

nshots = [
    [],
    [""],
    ["",""],
    ["","","","",""],
    []
]

